In [2]:
# Import libraries

import config
import MySQLdb
import sshtunnel
import pandas as pd
from backtesting import Backtest, Strategy
import backtrader as bt
import talib

sshtunnel.SSH_TIMEOUT = 300.0
sshtunnel.TUNNEL_TIMEOUT = 300.0

/home/mgallagherdata/.virtualenvs/myvirtualenv/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
# Query and read in the SQL table to a df

query = """
SELECT *
FROM daily_sp500
WHERE adv > 2000000 AND (date > '2016-01-20' AND date < '2018-01-01');
"""

with sshtunnel.SSHTunnelForwarder(
    (config.ssh_host),
    ssh_username=config.user,
    ssh_password=config.user_passwd,
    remote_bind_address=(config.host, 3306)
) as tunnel:
    connection = MySQLdb.connect(
        user=config.user,
        passwd=config.db_passwd,
        host='127.0.0.1', port=tunnel.local_bind_port,
        db=config.db,
    )

    df = pd.read_sql(query, connection)
    
df.columns = df.columns.str.title()
df = df[df['Symbol'].isin(['AAPL', 'TSLA', 'NVDA', 'CRM', 'NFLX', 'META'])]

In [4]:
# Define the strategy
class CustomLongStrategy(Strategy):
    def init(self):
        self.ema9 = self.I(lambda: self.data.df['Ema_9'], name='EMA-9')
        self.rsi = self.I(talib.RSI, self.data.df['Close'], 14)
        
    def next(self):
        current_idx = len(self.data) - 1  
        # Check the conditions for a long entry
        if (
            self.data.df['Range_Pct'].iloc[current_idx] > 0.8 and  # Current day Range_Pct > 0.8
            self.data.df['Rvol'].iloc[current_idx] > 1 and          # Current day Rvol > 1
            self.data.df['Adv'].iloc[current_idx] > 2000000 and     # Current day Adv > 2,000,000
            self.data.df['Pct_Gap'].iloc[current_idx] > 0 and      # Current day Pct_Gap > 0
            self.data.Open[current_idx] > self.data.df['Ema_9'].iloc[current_idx]  # Open > 9_Ema
        ):
            current_rvol = self.data.df['Rvol'].iloc[current_idx]
            self.buy()

        for trade in self.trades:
            if trade.is_long:
                entry_bar_low = self.data.df['Low'].iloc[trade.entry_bar]
                current_bar_low = self.data.df['Low'].iloc[current_idx]
                if current_bar_low < entry_bar_low:
                    trade.close()
                # Exit after 8 bars
                else:
                    if len(self.data) - trade.entry_bar >= 8:
                        trade.close()

In [5]:
# Perform the backtest on multiple symbols

# Get a list of unique symbols from the DataFrame
symbols = df['Symbol'].unique()

# Dictionary to hold the results for each symbol
results = {}

# Run the backtest for each symbol and store the results
for symbol in symbols:
    print(f"Running backtest for {symbol}...")
    
    # Filter the df for the current symbol
    symbol_df = df[df['Symbol'] == symbol]
    
    # Run the backtest
    bt = Backtest(symbol_df, CustomLongStrategy, cash=10000, commission=.002)
    stats = bt.run()
    
    # Print the stats
    print(stats)
    
    # Store the stats in the results dictionary
    results[symbol] = stats

results_df = pd.DataFrame.from_dict(results, orient='index')

# Display aggregated results
print(results_df.to_csv('backtest.csv'))

Running backtest for AAPL...
Start                                   751.0
End                                    1240.0
Duration                                489.0
Exposure Time [%]                   17.755102
Equity Final [$]                   9935.26302
Equity Peak [$]                   10227.59902
Return [%]                           -0.64737
Buy & Hold Return [%]                66.86058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.988657
Avg. Drawdown [%]                   -8.988657
Max. Drawdown Duration                  460.0
Avg. Drawdown Duration                  460.0
# Trades                                 16.0
Win Rate [%]                            31.25
Best Trade [%]                        6.63279
Worst Trade [%]                     -2.640517
Avg. 

In [11]:
# Perform the backtest
bt = Backtest(df, CustomLongStrategy, cash=10000, commission=.002)
stats = bt.run()

from bokeh.io.export import export_png
from bokeh.plotting import output_file, save

In [15]:
plot = bt.plot(open_browser=False)
save(plot)

'/home/mgallagherdata/backtesting/exports/backtest_plot.png'

In [ ]:
print(stats['_trades'].sort_values('ReturnPct', ascending=False))

In [ ]:
# Function to run backtest for each symbol

def backtest_symbol(symbol, df):
    # Filter data for the current symbol
    df_symbol = df[df['Symbol'] == symbol].copy()

    # Perform the backtest for this symbol
    bt = Backtest(df_symbol, CustomSmaCross, cash=10000, commission=.002)
    stats = bt.run()
    
    return stats

# Get a list of unique symbols from the DataFrame
symbols = df['Symbol'].unique()

# Dictionary to hold the results for each symbol
results = {}

# Run the backtest for each symbol and store the results
for symbol in symbols:
    print(f"Running backtest for {symbol}...")
    stats = backtest_symbol(symbol, df)
    results[symbol] = stats
    print(stats)

# Access the results for a specific symbol
print("AAPL backtest results:")
print(results['AAPL'])

# Plot results for the symbol
bt.plot() 